In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import os.path
import pretty_midi
import pickle
import random

### Evaluate system performance

In [3]:
def readGroundTruthLabels(gtfile):
    d={}
    d1={}
    with open(gtfile, "r") as f:
        for line in f:
            data = line.split(',')
            key = data[0]
            d[key] = []
            d1[key] = key
            for idx,item in enumerate(data):
                if idx != 0 and item != "x":
                    try:
                        pieceNum = int(item)
                        d[key].append(pieceNum)
                    except:
                        pass
                    try:
                        pieceNum = int(item)
                        d1[str(pieceNum)] = key
                    except:
                        pass
    return d, d1

In [4]:
def readHypothesisFiles(hypdir, benchmark):
    l = []
    if benchmark == 1:
        for hypfile in sorted(glob.glob("{}/*.pkl".format(hypdir))):
            with open(hypfile, "rb") as f:
                l.append([os.path.splitext(os.path.basename(hypfile))[0],pickle.load(f)])
    elif benchmark == 0 or benchmark == 2:
        for hypfile in sorted(glob.glob("{}/*.hyp".format(hypdir))):
            print(hypfile)
            with open(hypfile, "rb") as f:
                l.append(pickle.load(f))
    elif benchmark == 3:
        for hypfile in sorted(glob.glob("{}/*.hyp".format(hypdir))):
            print(hypfile)
            with open(hypfile, "rb") as f:
                data = pickle.load(f)
                pieceScores = [(x[1],x[0]) for x in data[1]]
                l.append((data[0],pieceScores))
    return l

In [5]:
s = set([])
def collapseIds(pieceScores):
    pieceScores = list(dict.fromkeys(pieceScores))
    return pieceScores
        

In [6]:
def getRank(pieceScores,gt,idt,queryid,benchmark, condition):
    query = queryid.split('_')[0]
    l = []
    count = 0
    if benchmark:
        pieceScores = collapseIds(pieceScores)
    rank = -1
    print(pieceScores[:5])
    for i in range(len(pieceScores)):
        if benchmark == 1 or benchmark == 2:
            num = pieceScores[i].split("_")[0]
            if 'p' in num:
                if condition == 2:
                    continue
                if query.split("_")[0] == num:
                    rank = count+1
                    return rank
            elif int(num) in gt[query]:
                rank = count+1
                return rank
            count = count+1            
        else:
            if pieceScores[i][0][0]=='p':
                if condition == 2:
                        continue
                if(pieceScores[i][0]==queryid.split('_')[0]):
                    rank=count+1
                    return rank
                if not pieceScores[i][0] in idt.keys():
                    count+=1
                elif not idt[pieceScores[i][0]] in l:
                    count+=1
                    l.append(idt[idt[pieceScores[i][0]]])
            else:
                try:
                    if (int(pieceScores[i][0].split("**")[-1]) in gt[query]):
                        rank = count+1
                        return rank

                    if not str(int(pieceScores[i][0].split("**")[-1])) in idt.keys():
                        count+=1
                    elif not idt[str(int(pieceScores[i][0].split("**")[-1]))] in l:
                        count+=1
                        l.append(idt[str(int(pieceScores[i][0].split("**")[-1]))])
                except:
                    pass
    return rank

In [7]:
def calcPrecisionRecall(hypdir, gtfile, benchmark = False, condition = 1):
    hyps = readHypothesisFiles(hypdir, benchmark)
    gt,idt = readGroundTruthLabels(gtfile)
    MRR = 0
    runtimes = []
    MRRs = []
    if condition == 2:
        valid = []
        with open('condition2.txt','r') as f:
            for line in f:
                valid.append(line.strip())
                
    hyps = sorted(hyps, key = lambda x: int(x[0].split("_")[0][1:])*100+int(x[0].split("_")[1][1:]))
    count = 0
    if len(hyps[0])==2:
        for queryid, pieceScores in hyps:
            print(pieceScores[:5])
            if condition == 2:
                if not queryid.split('_')[0].strip() in valid:
                    count+=1
                    continue
            rank = getRank(pieceScores,gt,idt,queryid,benchmark, condition)
            MRRs.append((queryid,rank))
            print(MRRs[-1])
            if rank != -1:
                MRR=MRR+1/(rank)
    else:
        for queryid, pieceScores, runtime in hyps:
            print(pieceScores[:5])
            if condition == 2:
                if not queryid.split('_')[0].strip() in valid:
                    count+=1
                    continue
            rank = getRank(pieceScores,gt,idt,queryid,benchmark, condition)
            MRRs.append((queryid,rank))
            print(MRRs[-1])
            if rank != -1:
                MRR=MRR+1/(rank)

    return MRR/len(MRRs),MRRs, runtimes

In [8]:
def calcOverlap(seg1, seg2):
    overlap_lb = max(seg1[0], seg2[0])
    overlap_ub = min(seg1[1], seg2[1])
    overlap = np.clip(overlap_ub - overlap_lb, 0, None)
    return overlap    

In [11]:
hypdir = '/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp'
#hypdir = '/home/dyang/old/SheetMusicID/experiments/AryaDynamicConfirm/hyp'
condition2 = "condition2.txt"

#0 is normal
#1 is old paper
#2 is new paper
#3 is weird test
benchmark = 0
queryGTFile = 'piece_To_id.csv'
MRR = 0
MRR, MRRs, runtimes = calcPrecisionRecall(hypdir, queryGTFile, benchmark, condition = 1)

/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p100_q1.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p100_q10.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p100_q2.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p100_q3.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p100_q4.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p100_q5.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p100_q6.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p100_q7.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p100_q8.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p100_q9.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p102_q1.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p102_q10.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/t

/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p112_q6.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p112_q7.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p112_q8.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p112_q9.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p113_q1.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p113_q10.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p113_q2.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p113_q3.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p113_q4.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p113_q5.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p113_q6.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p113_q7.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/te

/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p127_q1.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p127_q10.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p127_q2.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p127_q3.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p127_q4.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p127_q5.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p127_q6.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p127_q7.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p127_q8.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p127_q9.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p128_q1.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p128_q10.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/t

/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p139_q9.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p13_q1.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p13_q10.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p13_q2.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p13_q3.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p13_q4.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p13_q5.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p13_q6.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p13_q7.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p13_q8.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p13_q9.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p140_q1.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_

/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p152_q4.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p152_q5.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p152_q6.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p152_q7.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p152_q8.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p152_q9.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p153_q1.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p153_q10.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p153_q2.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p153_q3.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p153_q4.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/test_N_GRAM_2/hyp/p153_q5.hyp
/home/dyang/SheetMidiSearchRetrieval/experiments/te

...

('p76_q10', 7109)
[('p77', 188), ('dKnuth,_J%C3%BCrgenPictures_at_an_Exhibition_**41547', 84), ('dMussorgsky,_ModestPictures_at_an_Exhibition_**41547', 84), ('dMussorgsky,_ModestPictures_at_an_Exhibition_**85544', 83), ('dKnuth,_J%C3%BCrgenPictures_at_an_Exhibition_**01571', 83)]
[('p77', 188), ('dKnuth,_J%C3%BCrgenPictures_at_an_Exhibition_**41547', 84), ('dMussorgsky,_ModestPictures_at_an_Exhibition_**41547', 84), ('dMussorgsky,_ModestPictures_at_an_Exhibition_**85544', 83), ('dKnuth,_J%C3%BCrgenPictures_at_an_Exhibition_**01571', 83)]
('p77_q1', 1)
[('p77', 115), ('dKnuth,_J%C3%BCrgenPictures_at_an_Exhibition_**41547', 62), ('dMussorgsky,_ModestPictures_at_an_Exhibition_**41547', 62), ('dKnuth,_J%C3%BCrgenPictures_at_an_Exhibition_**107727', 61), ('dKnuth,_J%C3%BCrgenPictures_at_an_Exhibition_**392723', 61)]
[('p77', 115), ('dKnuth,_J%C3%BCrgenPictures_at_an_Exhibition_**41547', 62), ('dMussorgsky,_ModestPictures_at_an_Exhibition_**41547', 62), ('dKnuth,_J%C3%BCrgenPictures_at_an_Ex

[('dDebussy,_ClaudeChildren%27s_Corner_**82215', 55), ('dGouin,_PierreChildren%27s_Corner_**82215', 55), ('dGouin,_PierreChildren%27s_Corner_**252525', 53), ('dDebussy,_ClaudeChildren%27s_Corner_**252525', 53), ('dDebussy,_ClaudeChildren%27s_Corner_**00505', 52)]
[('dDebussy,_ClaudeChildren%27s_Corner_**82215', 55), ('dGouin,_PierreChildren%27s_Corner_**82215', 55), ('dGouin,_PierreChildren%27s_Corner_**252525', 53), ('dDebussy,_ClaudeChildren%27s_Corner_**252525', 53), ('dDebussy,_ClaudeChildren%27s_Corner_**00505', 52)]
('p126_q6', 1)
[('dGouin,_PierreChildren%27s_Corner_**252525', 90), ('dDebussy,_ClaudeChildren%27s_Corner_**252525', 90), ('p126', 89), ('dDebussy,_ClaudeChildren%27s_Corner_**252526', 88), ('dDebussy,_ClaudeChildren%27s_Corner_**00505', 88)]
[('dGouin,_PierreChildren%27s_Corner_**252525', 90), ('dDebussy,_ClaudeChildren%27s_Corner_**252525', 90), ('p126', 89), ('dDebussy,_ClaudeChildren%27s_Corner_**252526', 88), ('dDebussy,_ClaudeChildren%27s_Corner_**00505', 88)]
(

('p159_q2', 12872)
[('dCommette,_%C3%89douard6_Pieces_**510750', 29), ('dWheeldon,_Herbert_ArthurCanzona_**386953', 28), ('dLef%C3%A9bure-W%C3%A9ly,_Louis_James_AlfredL%27Organiste_moderne_**17640', 22), ('dBeethoven,_Ludwig_vanPiano_Sonata_No.16,_Op.31_No.1_**419637', 20), ('dBeethoven,_Ludwig_vanPiano_Sonata_No.16,_Op.31_No.1_**03169', 19)]
[('dCommette,_%C3%89douard6_Pieces_**510750', 29), ('dWheeldon,_Herbert_ArthurCanzona_**386953', 28), ('dLef%C3%A9bure-W%C3%A9ly,_Louis_James_AlfredL%27Organiste_moderne_**17640', 22), ('dBeethoven,_Ludwig_vanPiano_Sonata_No.16,_Op.31_No.1_**419637', 20), ('dBeethoven,_Ludwig_vanPiano_Sonata_No.16,_Op.31_No.1_**03169', 19)]
('p159_q3', 8)
[('p159', 135), ('dDebussy,_ClaudeKhamma_**14240', 33), ('dCzerny,_Carl100_Progressive_Studies,_Op.139_**240955', 32), ('dKnuth,_J%C3%BCrgenPictures_at_an_Exhibition_**10719', 32), ('dMussorgsky,_ModestPictures_at_an_Exhibition_**10719', 32)]
[('p159', 135), ('dDebussy,_ClaudeKhamma_**14240', 33), ('dCzerny,_Carl

('p164_q9', -1)
[('p164', 9), ('dJensen,_AdolfWanderbilder,_Op.17_**447408', 5), ('dPrudent,_%C3%89mileEtudes-Lieder,_Op.60_**283486', 5), ('dGurlitt,_CorneliusT%C3%A4gliche_Studien,_Op.91_**373943', 4), ('dLiszt,_FranzHungarian_Rhapsody_No.10,_S.244**00378', 4)]
[('p164', 9), ('dJensen,_AdolfWanderbilder,_Op.17_**447408', 5), ('dPrudent,_%C3%89mileEtudes-Lieder,_Op.60_**283486', 5), ('dGurlitt,_CorneliusT%C3%A4gliche_Studien,_Op.91_**373943', 4), ('dLiszt,_FranzHungarian_Rhapsody_No.10,_S.244**00378', 4)]
('p164_q10', 1)
[('p166', 174), ('dCzerny,_CarlThe_School_of_Velocity,_Op.299_**01002', 60), ('dCzerny,_Carl125_Passagen%C3%BCbungen,_Op.261_**245853', 53), ('dMaylath,_HenryNew_Graded_Piano_Method_**82021', 53), ('dNeustedt,_CharlesExercice_gymnastique_**400161', 51)]
[('p166', 174), ('dCzerny,_CarlThe_School_of_Velocity,_Op.299_**01002', 60), ('dCzerny,_Carl125_Passagen%C3%BCbungen,_Op.261_**245853', 53), ('dMaylath,_HenryNew_Graded_Piano_Method_**82021', 53), ('dNeustedt,_CharlesE

('p199_q9', -1)
[('p199', 56), ('dRosellen,_Henri3_R%C3%AAveries,_Op.31_**245470', 17), ('dHeins,_CarlChicagoer_Weltausstellungs-Gavotte,_Op.130_**409970', 16), ('dRosellen,_Henri3_R%C3%AAveries,_Op.31_**112878', 16), ('dRosellen,_Henri3_R%C3%AAveries,_Op.31_**245471', 15)]
[('p199', 56), ('dRosellen,_Henri3_R%C3%AAveries,_Op.31_**245470', 17), ('dHeins,_CarlChicagoer_Weltausstellungs-Gavotte,_Op.130_**409970', 16), ('dRosellen,_Henri3_R%C3%AAveries,_Op.31_**112878', 16), ('dRosellen,_Henri3_R%C3%AAveries,_Op.31_**245471', 15)]
('p199_q10', 1)
[('dGrieg,_EdvardLyric_Pieces,_Op.47_**00182', 61), ('p200', 54), ('dGrieg,_EdvardLyric_Pieces,_Op.47_**285709', 50), ('dGrieg,_EdvardLyric_Pieces,_Op.47_**30365', 49), ('dStojowski,_Sigismond2_Orientales,_Op.10_**503862', 18)]
[('dGrieg,_EdvardLyric_Pieces,_Op.47_**00182', 61), ('p200', 54), ('dGrieg,_EdvardLyric_Pieces,_Op.47_**285709', 50), ('dGrieg,_EdvardLyric_Pieces,_Op.47_**30365', 49), ('dStojowski,_Sigismond2_Orientales,_Op.10_**503862',

In [12]:
MRR

0.8452127780911072

### Investigate Errors

In [23]:
def printDebuggingInfo(MRRs):
    for i, (queryid, rank) in enumerate(MRRs):
        print(queryid, rank)

In [24]:
printDebuggingInfo(MRRs)

p1_q1 1
p1_q2 1
p1_q3 1
p1_q4 1
p1_q5 1
p1_q6 1
p1_q7 1
p1_q8 1
p1_q9 1
p1_q10 1
p5_q1 1
p5_q2 1
p5_q3 1
p5_q4 1
p5_q5 1
p5_q6 1
p5_q7 1
p5_q8 1
p5_q9 1
p5_q10 1
p11_q1 1
p11_q2 1
p11_q3 1
p11_q4 1
p11_q5 1
p11_q6 1
p11_q7 1
p11_q8 1
p11_q9 1
p11_q10 1
p15_q1 1
p15_q2 1
p15_q3 1
p15_q4 1
p15_q5 1
p15_q6 1
p15_q7 1
p15_q8 1
p15_q9 1
p15_q10 1
p21_q1 1
p21_q2 1
p21_q3 1
p21_q4 1
p21_q5 300000
p21_q6 1
p21_q7 1
p21_q8 1
p21_q9 1
p21_q10 1
p25_q1 1
p25_q2 1
p25_q3 1
p25_q4 1
p25_q5 1
p25_q6 1
p25_q7 1
p25_q8 1
p25_q9 1
p25_q10 1
p31_q1 1
p31_q2 1
p31_q3 1
p31_q4 1
p31_q5 1
p31_q6 1
p31_q7 1
p31_q8 1
p31_q9 1
p31_q10 1
p35_q1 1
p35_q2 1
p35_q3 1
p35_q4 1
p35_q5 1
p35_q6 1
p35_q7 1
p35_q8 1
p35_q9 1
p35_q10 1
p41_q1 513
p41_q2 1
p41_q3 1
p41_q4 1
p41_q5 1
p41_q6 1
p41_q7 1
p41_q8 1
p41_q9 1
p41_q10 1
p45_q1 1
p45_q2 1
p45_q3 1
p45_q4 1
p45_q5 1
p45_q6 1
p45_q7 1
p45_q8 1
p45_q9 1
p45_q10 1
p51_q1 1
p51_q2 1
p51_q3 1
p51_q4 1
p51_q5 1
p51_q6 1
p51_q7 1
p51_q8 1
p51_q9 1
p51_q10 300000
p55_q1 

### Measure Runtime

In [22]:
def showRuntimeStats(durs):
    durs = []
    durs = np.array(durs)
    avgDur = np.mean(durs)
    minDur = np.min(durs)
    maxDur = np.max(durs)
    stdDur = np.std(durs)
    print('Avg Duration: {:.2f} sec'.format(avgDur))
    print('Std Duration: {:.2f} sec'.format(stdDur))
    print('Min Duration: {:.2f} sec'.format(minDur))
    print('Max Duration: {:.2f} sec'.format(maxDur))
    plt.hist(durs, bins=np.arange(0,2,.1))
    plt.xlabel('Runtime (sec)')
    plt.ylabel('Count')

In [23]:
showRuntimeStats(runtimes)

ValueError: could not convert string to float: '_S.171_**423369q\x04Ke\x86q\x05X(\x00\x00\x00dLiszt'